# Day 17

## part 1

- 3-bit computer with 3 registers, each which can hold any integer
- 8 instructions:
- instruction pointer starts at 0 and increments by 2 when not jumping
- program halts the counter moves past the end of the program
- operands can be literal or combo
    - literal gives the value itself
    - combo operands 0-3 are literals
    - combo operands 4-6 map to `A`, `B`, `C`
    - combo operand 7 is unot used

Here are the opcodes

|code|name|function|stored to|
|---|---|---|---|
|0|adv|divides the `A` register by $2^{x}$ where x is the combo operand, truncated to an int|`A`|
|1|bxl|bitwise XOR of register B and the literal operand|(`B` % 8)|
|2|bst|combo operand % 8|`B`|
|3|jnz|nop if `A`==0. If `A`!=0 jumps to literal operand and instruction pointer is not incremented this time| |
|4|bxc|bitwise XOR of `B` and `C`|`B`|
|5|out|combo operand % 8|output|
|6|bdv|as adv but for `B`|`B`|
|7|cdv|as adv but for `C`|`C`|


- Determine the program output as comma delimited values


In [46]:
from dataclasses import dataclass
import logging

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 17')
logging.basicConfig(level=logging.INFO)

In [47]:
@dataclass
class Computer:
    A: int
    B: int
    C: int
    programme: list[int]
    pointer: int = 0

    def step(self) -> list[int] | None:
        """runs the next programme step"""
        log.debug(f'{self}')
        if self.pointer >= len(self.programme):
            log.info('halt')
            return None
        opcode = self.programme[self.pointer]
        operand = self.programme[self.pointer + 1]
        log.debug(f'{opcode=}, {operand=}')
        out = []
        match opcode:
            case 0:
                self.adv(operand)
                self.increment()
            case 1:
                self.bxl(operand)
                self.increment()
            case 2:
                self.bst(operand)
                self.increment()
            case 3:
                self.jnz(operand)
            case 4:
                self.bxc(operand)
                self.increment()
            case 5:
                out.append(self.out(operand))
                self.increment()
            case 6:
                self.bdv(operand)
                self.increment()
            case 7:
                self.cdv(operand)
                self.increment()
            case _:
                raise ValueError(f'opcode not recognised: {self}')
        return out
    
    def increment(self) -> None:
        """increments the programme pointer"""
        self.pointer += 2
        log.debug('pointer: +2')

    def combo(self, operand: int) -> int:
        """returns the combo operand value"""
        reg_map = {4: self.A, 5: self.B, 6: self.C}
        if operand in [0, 1, 2, 3]:
            return operand
        elif operand in reg_map:
            return reg_map[operand]
        else:
            raise ValueError(f'Invalid combo operand: {self}')

    def adv(self, operand: int) -> None:
        numerator = self.A
        denominator = pow(2, self.combo(operand))
        self.A = numerator // denominator
        log.debug(f'adv: {self.A}->A')
    
    def bdv(self, operand: int) -> None:
        numerator = self.A
        denominator = pow(2, self.combo(operand))
        self.B = numerator // denominator
        log.debug(f'bdv: {self.B}->B')
    
    def cdv(self, operand: int) -> None:
        numerator = self.A
        denominator = pow(2, self.combo(operand))
        self.C = numerator // denominator
        log.debug(f'cdv: {self.C}->C')

    def bxl(self, operand: int) -> None:
        self.B ^= operand
        log.debug(f'bxl: {self.B}->B')
    
    def out(self, operand: int) -> int:
        value = self.combo(operand) % 8
        log.debug(f'out: {value}')
        return value
    
    def bst(self, operand: int) -> None:
        self.B = self.combo(operand) % 8
        log.debug(f'bst: {self.B}->B')
    
    def jnz(self, operand: int) -> None:
        if self.A == 0:
            log.debug('jnz: A=0')
            self.increment()
        else:
            log.debug(f'jnz: A!=0, pointer->{operand}')
            self.pointer = operand
    
    def bxc(self, *args) -> None:
        self.B ^= self.C
        log.debug(f'bxc: {self.B}->B')

parser = PC('\n\n', [
    PC('\n', PC(': ', [None, int])),  # registers
    PC(': ', [None, PC(',', int)])  # program
])
registers, programme = \
    parse_from_file('day_17_example.txt', parser, unnest_single_items=True)
computer = Computer(*registers, programme)

INFO:advent_of_code_utils.py:2 items loaded from "day_17_example.txt"


In [48]:
log.setLevel(logging.INFO)
log.debug(f'{computer=}')
output = []
while True:
    temp = computer.step()
    if temp is None:
        break
    else:
        output.extend(temp)
log.info(f'{output=}')

INFO:day 17:halt
INFO:day 17:output=[4, 6, 3, 5, 6, 3, 5, 2, 1, 0]


In [51]:
# great that looks like it works, lets run things for real
log.setLevel(logging.INFO)
registers, programme = \
    parse_from_file('day_17.txt', parser, unnest_single_items=True)
computer = Computer(*registers, programme)
output = []
while True:
    temp = computer.step()
    if temp is None:
        break
    else:
        output.extend(temp)

INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:day 17:halt


In [50]:
markdown(f'The programme output is: {','.join((str(v) for v in output))}')

The programme output is: 4,6,1,4,2,1,3,1,6

## part 2

- the programe is supposed to output another programme
- find the lowest initial value of A that produces the programme back

Ok so a few things stand out already
- the final instruction is `jnz(0)` so the program will only halt when `A=0`. Otherwise the programme pointer will be reset to 0.
- this is the only `jnz` opcode so the programme will always just run its entirity without any branches each iteration.
- The 2nd last instruction is `out(5)` which outputs `B` so again the behaviour should be pretty consistent.
- the 3rd last instruciton is `adv(3)` so `A` is divided by 8 each time

From this we can figure out the range of values for which `A` will ensure the correct number of outputs is added.

In [52]:
log.info(f'{len(computer.programme)=}')

INFO:day 17:len(computer.programme)=16


Since the programme is 16 digits long that means `A` must be != 0 for enough iterations to produce that many digits

In [60]:
for digit, power in zip(reversed(range(1, 17)), range(16)):
    print(f'{digit=}', f'A range = [{2**(power * 3)}, {2**((power + 1) * 3)})')

digit=16 A range = [1, 8)
digit=15 A range = [8, 64)
digit=14 A range = [64, 512)
digit=13 A range = [512, 4096)
digit=12 A range = [4096, 32768)
digit=11 A range = [32768, 262144)
digit=10 A range = [262144, 2097152)
digit=9 A range = [2097152, 16777216)
digit=8 A range = [16777216, 134217728)
digit=7 A range = [134217728, 1073741824)
digit=6 A range = [1073741824, 8589934592)
digit=5 A range = [8589934592, 68719476736)
digit=4 A range = [68719476736, 549755813888)
digit=3 A range = [549755813888, 4398046511104)
digit=2 A range = [4398046511104, 35184372088832)
digit=1 A range = [35184372088832, 281474976710656)


In [66]:
# lets try inside and outside that range of values to be sure
def load_and_run(initial_a: int = None) -> list[int]:
    """loads the programme and runs it swapping in an initial value of A"""

    registers, programme = \
        parse_from_file('day_17.txt', parser, unnest_single_items=True)
    computer = Computer(*registers, programme)
    if initial_a is not None:
        log.info(f'Set A to {initial_a}')
        computer.A = initial_a
    output = []
    while True:
        temp = computer.step()
        if temp is None:
            break
        else:
            output.extend(temp)
    return output

log.setLevel(logging.INFO)
for value in (2**(3*15) - 1, 2**(3*15), 2**(3*16) - 1, 2**(3*16)):
    output = load_and_run(value)
    log.info(f'{len(output)=}')
    log.info(f'{output=}')


INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:day 17:Set A to 35184372088831
INFO:day 17:halt
INFO:day 17:len(output)=15
INFO:day 17:output=[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2]
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:day 17:Set A to 35184372088832
INFO:day 17:halt
INFO:day 17:len(output)=16
INFO:day 17:output=[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5]
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:day 17:Set A to 281474976710655
INFO:day 17:halt
INFO:day 17:len(output)=16
INFO:day 17:output=[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2]
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:day 17:Set A to 281474976710656
INFO:day 17:halt
INFO:day 17:len(output)=17
INFO:day 17:output=[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5]


cool that narrows it down a bit. Next we just need to figure out what determines the value produced. We know its in `B` so let's see what happens to that register each iteration.
The instructions that affect `B` are:
- `bxl` (1)
- `bst` (2)
- `bxc` (4)
- `bdv` (6)

So going backwards the instructions that affect the output are:
- `bxl(4)`
- `bxc`
- `cdv(5)` so `A` // $2^{C}$ -> C.
- `bxl(1)` - `B` ^= 1
- `bst(4)` - `B` = `A` % 8

`bdv` appears to never be executed

In [69]:
# log.setLevel(logging.INFO)

# output = load_and_run(2**(3*15))
# log.info(f'{output=}')
print(computer.programme)

[2, 4, 1, 1, 7, 5, 4, 6, 1, 4, 0, 3, 5, 5, 3, 0]


So let's work backwards
- out = (`B` % 8)
- out = ((`B` % 8) ^ 4)
- out = (((`B` % 8) ^ 4) ^ `C`)
- out = (((`B` % 8) ^ 4) ^ (`A` // $2^C$))
- out = ((((`B` % 8) ^ 1) ^ 4) ^ (`A` // $2^C$))
- out = ((((`A` % 8) ^ 1) ^ 4) ^ (`A` // $2^C$))

cool so `B` gets completely overwritten by `A` each time and is always set to a value between 0-7 by the first opcode so the

From some testing it looks like adding values to different orders of magnitude affects specific digits - which makes sense as lower orders of magnitude are truncated earlier in the iterations.
So we should be able to programatically solve by incrementing a set of offsets applied by increasing powers of $2^3$ until we get the required programme digit

In [105]:
def find_a_value() -> int:
    """returns the value required to produce the programme back as output"""
    offsets = [0]*len(computer.programme)
    for digit in reversed(range(len(computer.programme))):
        while True:
            log.warning(f'trying {digit=}: {offsets[digit]=}')
            start = 2**(3 * (len(computer.programme) - 1))
            for index, value in enumerate(offsets):
                start += value * 2**(3*index)

            output = load_and_run(start)
            log.info(f'{output=}')
            if output[digit] == computer.programme[digit]:
                break
            # else
            offsets[digit] += 1
            if offsets[digit] > 2**3:
                log.error('couldn\'t find an offset')
                return None
    start = 2**(3 * (len(computer.programme) - 1))
    for index, value in enumerate(offsets):
        start += value * 2**(3*index)
    log.warning(f'succeeded with {start=}')
    return start

log.setLevel(logging.WARNING)
a_value = find_a_value()

INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:adv

In [106]:
markdown(f'the A value required is: {a_value}')

the A value required is: 202366627359274